In [1]:
from pprint import pprint as pp
from tqdm.notebook import tqdm, trange
import time
import entrezpy.esummary.esummarizer
import re
import itertools
import json

In [2]:
with open(r"D:\ncbi_dataset\ncbi_dataset\data\sampleNames.txt", "r") as f:
    sampleHeaders = f.readlines()
n_samples = len(sampleHeaders)

In [16]:
lookedup = []
lookedup_filepath = r"C:\Users\Konrad Grudzinski\OneDrive - University of Glasgow\Computing\4th Year\Individual Project\Source\covid19\lookedup_samples.txt"
samples = {}
samples_filepath = r"C:\Users\Konrad Grudzinski\OneDrive - University of Glasgow\Computing\4th Year\Individual Project\Source\covid19\samples.json"
grouping = 500

In [17]:
def get_data(headers, samples, lookedup):
    sampleNames = list(map(lambda h: re.findall(">(.*\.[0-9])", h)[0], headers))
    e = entrezpy.esummary.esummarizer.Esummarizer("L4project", "2408448g@student.gla.ac.uk")
    analyzer = e.inquire({'db' : 'nuccore', 'id' : sampleNames})
    summaries = analyzer.get_result().summaries
    for value in summaries.values():
        if "accessionversion" in value and value["accessionversion"] in sampleNames:
            types = value["subtype"].split("|")
            names = value["subname"].split("|")
            d = {}
            for i in range(len(types)):
                if types[i] == "collection_date":
                    d["collection_date"] = names[i]
                elif types[i] == "country":
                    d["country"] = names[i]
            samples[value["accessionversion"]] = d
    lookedup.extend(sampleNames)
    return samples, lookedup

In [24]:
for headers in tqdm(list(zip(*[iter(sampleHeaders)]*grouping))):
    samples, lookedup = get_data(headers, samples, lookedup)
samples, lookedup = get_data(sampleHeaders[(n_samples - n_samples%grouping):], samples, lookedup)

  0%|          | 0/271 [00:00<?, ?it/s]

In [25]:
with open(lookedup_filepath, "w") as f:
    f.write("\n".join(lookedup)+"\n")

In [26]:
with open(samples_filepath, "w") as f:
    json_object = json.dumps(samples, indent = 4) 
    f.write(json_object)

In [29]:
len(samples)

702227